# Filtro
Bloque diseñado para obtener la lista de tablas relevantes para la construcción de la consulta SQL

### Librerías necesarias

In [ ]:
import sys
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Conexión a DB local

In [ ]:
# Base de datos dummy para hacer las pruebas.
from langchain.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite://///home/llmuser/DB/laloss2.db",sample_rows_in_table_info=0)

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

### Carga del modelo Mixtral en LlamaCpp

In [ ]:
# Modelos con llama_cpp
llm = LlamaCpp(
    model_path="dockerFolder/mixtral-8x7b-v0.1.Q5_K_M.gguf",
    temperature=0.0,
    n_gpu_layers=-1,
    max_tokens=1000,
    n_ctx=1024*10,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

## Prompt con few-shot-learning and chain-of-thought

In [ ]:
#Prompt con few-shot-learning and chain-of-thought
tables_prompt = """
tables_prompt = """
This is the DB information:
CREATE TABLE "ACCOUNT" (
	"ACCOUNT_ID" INTEGER, 
	"ACCOUNT_TYPE" TEXT NOT NULL, 
	"ACCOUNT_STATUS" INTEGER NOT NULL, 
	"CURRENCY" TEXT NOT NULL, 
	"CREATE_DATE" TEXT NOT NULL, 
	"CLOSE_DATE" TEXT, 
	"AMOUNT" INTEGER NOT NULL, 
	"LAST_TRANSACTION" TEXT, 
	"FK_ACCOUNT_USER_ID" INTEGER, 
	PRIMARY KEY ("ACCOUNT_ID"), 
	FOREIGN KEY("FK_ACCOUNT_USER_ID") REFERENCES "USERS" ("USER_ID")
)


CREATE TABLE "ADDRESS" (
	"ADDRESS_ID" INTEGER, 
	"STREET" TEXT NOT NULL, 
	"CITY" TEXT NOT NULL, 
	"STATE" TEXT NOT NULL, 
	"POSTAL_CODE" TEXT NOT NULL, 
	"COUNTRY" TEXT NOT NULL, 
	PRIMARY KEY ("ADDRESS_ID")
)


CREATE TABLE "DOCUMENT" (
	"ID_DOCUMENT" INTEGER, 
	"NUM_DOCUMENT" TEXT NOT NULL, 
	"DOCUMENT_TYPE" TEXT NOT NULL, 
	"ISSUE_DATE" TEXT NOT NULL, 
	"EXP_DATE" TEXT NOT NULL, 
	PRIMARY KEY ("ID_DOCUMENT"), 
	UNIQUE ("ID_DOCUMENT", "NUM_DOCUMENT", "DOCUMENT_TYPE")
)


CREATE TABLE "EMPLOYEE" (
	"EMPLOYEE_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"FK_EMPLOYEE_ADDRESS" INTEGER, 
	"FK_EMPLOYEE_ROLE_ID" INTEGER, 
	PRIMARY KEY ("EMPLOYEE_ID"), 
	FOREIGN KEY("FK_EMPLOYEE_ROLE_ID") REFERENCES "ROLE" ("ROLE_ID"), 
	FOREIGN KEY("FK_EMPLOYEE_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "ENTITIE" (
	"ENTITIE_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"RUC" INTEGER NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"E_ADDRESS" INTEGER, 
	"BUSINESS_TYPE" TEXT NOT NULL, 
	"FK_OWNERSHIP" INTEGER, 
	PRIMARY KEY ("ENTITIE_ID"), 
	FOREIGN KEY("FK_OWNERSHIP") REFERENCES "P_PERSON" ("PERSON_ID"), 
	FOREIGN KEY("E_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "PROFESSION" (
	"PROFESSION_ID" INTEGER, 
	"NAME_PROFESSION" TEXT NOT NULL, 
	"DESCRIPTION" TEXT NOT NULL, 
	"PROFESSION_RANKINGS" INTEGER NOT NULL, 
	PRIMARY KEY ("PROFESSION_ID")
)


CREATE TABLE "P_PERSON" (
	"PERSON_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"SURNAME" TEXT NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"FK_P_ADDRESS" INTEGER, 
	"FK_P_PROFESSION_ID" INTEGER, 
	"FK_P_DOCUMENT" INTEGER, 
	PRIMARY KEY ("PERSON_ID"), 
	FOREIGN KEY("FK_P_DOCUMENT") REFERENCES "DOCUMENT" ("ID_DOCUMENT"), 
	FOREIGN KEY("FK_P_PROFESSION_ID") REFERENCES "PROFESSION" ("PROFESSION_ID"), 
	FOREIGN KEY("FK_P_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "REQUEST_CHECKBOOKS" (
	"REQUEST_id" INTEGER, 
	"FK_ACCOUNT_ID" INTEGER, 
	"REQUEST_STATE" TEXT NOT NULL, 
	"CREATE_DATE" TEXT NOT NULL, 
	PRIMARY KEY ("REQUEST_id"), 
	FOREIGN KEY("FK_ACCOUNT_ID") REFERENCES "ACCOUNT" ("ACCOUNT_ID")
)


CREATE TABLE "ROLE" (
	"ROLE_ID" INTEGER, 
	"DESCRIPTION" TEXT NOT NULL, 
	"PRIVILEGES" TEXT NOT NULL, 
	PRIMARY KEY ("ROLE_ID")
)


CREATE TABLE "USERS" (
	"USER_ID" INTEGER, 
	"USER_NAME" TEXT NOT NULL, 
	"FK_P_PERSON" INTEGER, 
	"FK_ENTITIE" INTEGER, 
	PRIMARY KEY ("USER_ID"), 
	FOREIGN KEY("FK_ENTITIE") REFERENCES "ENTITIE" ("ENTITIE_ID"), 
	FOREIGN KEY("FK_P_PERSON") REFERENCES "P_PERSON" ("PERSON_ID"), 
	CONSTRAINT chk_user CHECK (FK_P_PERSON = NULL AND FK_ENTITIE != NULL OR FK_P_PERSON != NULL AND FK_ENTITIE = NULL)
)

Q: "what is the role of user Gonzalo"
A: Let’s think step by step. In the question "what is the role of user Gonzalo", we are asked for user Gonzalo, the table USER not have an atribute ROLE, but EMPLOYEE has, and is a foreign_key from ROLE, so we need the tables EMPLOYEE and ROLE. 
Relevant_tables=["EMPLOYEE","ROLE"]


This is the DB information:
CREATE TABLE "ACCOUNT" (
	"ACCOUNT_ID" INTEGER, 
	"ACCOUNT_TYPE" TEXT NOT NULL, 
	"ACCOUNT_STATUS" INTEGER NOT NULL, 
	"CURRENCY" TEXT NOT NULL, 
	"CREATE_DATE" TEXT NOT NULL, 
	"CLOSE_DATE" TEXT, 
	"AMOUNT" INTEGER NOT NULL, 
	"LAST_TRANSACTION" TEXT, 
	"FK_ACCOUNT_USER_ID" INTEGER, 
	PRIMARY KEY ("ACCOUNT_ID"), 
	FOREIGN KEY("FK_ACCOUNT_USER_ID") REFERENCES "USERS" ("USER_ID")
)


CREATE TABLE "ADDRESS" (
	"ADDRESS_ID" INTEGER, 
	"STREET" TEXT NOT NULL, 
	"CITY" TEXT NOT NULL, 
	"STATE" TEXT NOT NULL, 
	"POSTAL_CODE" TEXT NOT NULL, 
	"COUNTRY" TEXT NOT NULL, 
	PRIMARY KEY ("ADDRESS_ID")
)


CREATE TABLE "DOCUMENT" (
	"ID_DOCUMENT" INTEGER, 
	"NUM_DOCUMENT" TEXT NOT NULL, 
	"DOCUMENT_TYPE" TEXT NOT NULL, 
	"ISSUE_DATE" TEXT NOT NULL, 
	"EXP_DATE" TEXT NOT NULL, 
	PRIMARY KEY ("ID_DOCUMENT"), 
	UNIQUE ("ID_DOCUMENT", "NUM_DOCUMENT", "DOCUMENT_TYPE")
)


CREATE TABLE "EMPLOYEE" (
	"EMPLOYEE_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"FK_EMPLOYEE_ADDRESS" INTEGER, 
	"FK_EMPLOYEE_ROLE_ID" INTEGER, 
	PRIMARY KEY ("EMPLOYEE_ID"), 
	FOREIGN KEY("FK_EMPLOYEE_ROLE_ID") REFERENCES "ROLE" ("ROLE_ID"), 
	FOREIGN KEY("FK_EMPLOYEE_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "ENTITIE" (
	"ENTITIE_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"RUC" INTEGER NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"E_ADDRESS" INTEGER, 
	"BUSINESS_TYPE" TEXT NOT NULL, 
	"FK_OWNERSHIP" INTEGER, 
	PRIMARY KEY ("ENTITIE_ID"), 
	FOREIGN KEY("FK_OWNERSHIP") REFERENCES "P_PERSON" ("PERSON_ID"), 
	FOREIGN KEY("E_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "PROFESSION" (
	"PROFESSION_ID" INTEGER, 
	"NAME_PROFESSION" TEXT NOT NULL, 
	"DESCRIPTION" TEXT NOT NULL, 
	"PROFESSION_RANKINGS" INTEGER NOT NULL, 
	PRIMARY KEY ("PROFESSION_ID")
)


CREATE TABLE "P_PERSON" (
	"PERSON_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"SURNAME" TEXT NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"FK_P_ADDRESS" INTEGER, 
	"FK_P_PROFESSION_ID" INTEGER, 
	"FK_P_DOCUMENT" INTEGER, 
	PRIMARY KEY ("PERSON_ID"), 
	FOREIGN KEY("FK_P_DOCUMENT") REFERENCES "DOCUMENT" ("ID_DOCUMENT"), 
	FOREIGN KEY("FK_P_PROFESSION_ID") REFERENCES "PROFESSION" ("PROFESSION_ID"), 
	FOREIGN KEY("FK_P_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "REQUEST_CHECKBOOKS" (
	"REQUEST_id" INTEGER, 
	"FK_ACCOUNT_ID" INTEGER, 
	"REQUEST_STATE" TEXT NOT NULL, 
	"CREATE_DATE" TEXT NOT NULL, 
	PRIMARY KEY ("REQUEST_id"), 
	FOREIGN KEY("FK_ACCOUNT_ID") REFERENCES "ACCOUNT" ("ACCOUNT_ID")
)


CREATE TABLE "ROLE" (
	"ROLE_ID" INTEGER, 
	"DESCRIPTION" TEXT NOT NULL, 
	"PRIVILEGES" TEXT NOT NULL, 
	PRIMARY KEY ("ROLE_ID")
)


CREATE TABLE "USERS" (
	"USER_ID" INTEGER, 
	"USER_NAME" TEXT NOT NULL, 
	"FK_P_PERSON" INTEGER, 
	"FK_ENTITIE" INTEGER, 
	PRIMARY KEY ("USER_ID"), 
	FOREIGN KEY("FK_ENTITIE") REFERENCES "ENTITIE" ("ENTITIE_ID"), 
	FOREIGN KEY("FK_P_PERSON") REFERENCES "P_PERSON" ("PERSON_ID"), 
	CONSTRAINT chk_user CHECK (FK_P_PERSON = NULL AND FK_ENTITIE != NULL OR FK_P_PERSON != NULL AND FK_ENTITIE = NULL)
)

Q: "what is the profesion of user Joaquin"
A: Let’s think step by step. In the question "what is the profesion of user Joaquin", we are asked for information about user Joaquin, the table USER have a foreign_keys from P_PERSON, and P_PERSON have an atribure PROFESSION that is a foreign_keys from PROFESSION, so we need the tables USER, P_PERSON and PROFESSION. 
Relevant_tables=["USERS","P_PERSON","PROFESSION"]


This is the DB information:
CREATE TABLE "ACCOUNT" (
	"ACCOUNT_ID" INTEGER, 
	"ACCOUNT_TYPE" TEXT NOT NULL, 
	"ACCOUNT_STATUS" INTEGER NOT NULL, 
	"CURRENCY" TEXT NOT NULL, 
	"CREATE_DATE" TEXT NOT NULL, 
	"CLOSE_DATE" TEXT, 
	"AMOUNT" INTEGER NOT NULL, 
	"LAST_TRANSACTION" TEXT, 
	"FK_ACCOUNT_USER_ID" INTEGER, 
	PRIMARY KEY ("ACCOUNT_ID"), 
	FOREIGN KEY("FK_ACCOUNT_USER_ID") REFERENCES "USERS" ("USER_ID")
)


CREATE TABLE "ADDRESS" (
	"ADDRESS_ID" INTEGER, 
	"STREET" TEXT NOT NULL, 
	"CITY" TEXT NOT NULL, 
	"STATE" TEXT NOT NULL, 
	"POSTAL_CODE" TEXT NOT NULL, 
	"COUNTRY" TEXT NOT NULL, 
	PRIMARY KEY ("ADDRESS_ID")
)


CREATE TABLE "DOCUMENT" (
	"ID_DOCUMENT" INTEGER, 
	"NUM_DOCUMENT" TEXT NOT NULL, 
	"DOCUMENT_TYPE" TEXT NOT NULL, 
	"ISSUE_DATE" TEXT NOT NULL, 
	"EXP_DATE" TEXT NOT NULL, 
	PRIMARY KEY ("ID_DOCUMENT"), 
	UNIQUE ("ID_DOCUMENT", "NUM_DOCUMENT", "DOCUMENT_TYPE")
)


CREATE TABLE "EMPLOYEE" (
	"EMPLOYEE_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"FK_EMPLOYEE_ADDRESS" INTEGER, 
	"FK_EMPLOYEE_ROLE_ID" INTEGER, 
	PRIMARY KEY ("EMPLOYEE_ID"), 
	FOREIGN KEY("FK_EMPLOYEE_ROLE_ID") REFERENCES "ROLE" ("ROLE_ID"), 
	FOREIGN KEY("FK_EMPLOYEE_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "ENTITIE" (
	"ENTITIE_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"RUC" INTEGER NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"E_ADDRESS" INTEGER, 
	"BUSINESS_TYPE" TEXT NOT NULL, 
	"FK_OWNERSHIP" INTEGER, 
	PRIMARY KEY ("ENTITIE_ID"), 
	FOREIGN KEY("FK_OWNERSHIP") REFERENCES "P_PERSON" ("PERSON_ID"), 
	FOREIGN KEY("E_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "PROFESSION" (
	"PROFESSION_ID" INTEGER, 
	"NAME_PROFESSION" TEXT NOT NULL, 
	"DESCRIPTION" TEXT NOT NULL, 
	"PROFESSION_RANKINGS" INTEGER NOT NULL, 
	PRIMARY KEY ("PROFESSION_ID")
)


CREATE TABLE "P_PERSON" (
	"PERSON_ID" INTEGER, 
	"NAME" TEXT NOT NULL, 
	"SURNAME" TEXT NOT NULL, 
	"PHONE_NUMBRE" TEXT NOT NULL, 
	"EMAIL" TEXT NOT NULL, 
	"FK_P_ADDRESS" INTEGER, 
	"FK_P_PROFESSION_ID" INTEGER, 
	"FK_P_DOCUMENT" INTEGER, 
	PRIMARY KEY ("PERSON_ID"), 
	FOREIGN KEY("FK_P_DOCUMENT") REFERENCES "DOCUMENT" ("ID_DOCUMENT"), 
	FOREIGN KEY("FK_P_PROFESSION_ID") REFERENCES "PROFESSION" ("PROFESSION_ID"), 
	FOREIGN KEY("FK_P_ADDRESS") REFERENCES "ADDRESS" ("ADDRESS_ID")
)


CREATE TABLE "REQUEST_CHECKBOOKS" (
	"REQUEST_id" INTEGER, 
	"FK_ACCOUNT_ID" INTEGER, 
	"REQUEST_STATE" TEXT NOT NULL, 
	"CREATE_DATE" TEXT NOT NULL, 
	PRIMARY KEY ("REQUEST_id"), 
	FOREIGN KEY("FK_ACCOUNT_ID") REFERENCES "ACCOUNT" ("ACCOUNT_ID")
)


CREATE TABLE "ROLE" (
	"ROLE_ID" INTEGER, 
	"DESCRIPTION" TEXT NOT NULL, 
	"PRIVILEGES" TEXT NOT NULL, 
	PRIMARY KEY ("ROLE_ID")
)


CREATE TABLE "USERS" (
	"USER_ID" INTEGER, 
	"USER_NAME" TEXT NOT NULL, 
	"FK_P_PERSON" INTEGER, 
	"FK_ENTITIE" INTEGER, 
	PRIMARY KEY ("USER_ID"), 
	FOREIGN KEY("FK_ENTITIE") REFERENCES "ENTITIE" ("ENTITIE_ID"), 
	FOREIGN KEY("FK_P_PERSON") REFERENCES "P_PERSON" ("PERSON_ID"), 
	CONSTRAINT chk_user CHECK (FK_P_PERSON = NULL AND FK_ENTITIE != NULL OR FK_P_PERSON != NULL AND FK_ENTITIE = NULL)
)

Q: "who is the owner of the company "Laloss SA"
A: Let’s think step by step. In the question "who is the owner of the company "Laloss SA", we are asked for information about a entitie call Laloss SA, the table ENTITIE have columns OWNERSHIP that is a foreign_keys from P_PERSON, so we need the tables ENTITIE and P_PERSON. 
Relevant_tables=["ENTITIE","P_PERSON"]

This is the DB information:
"""

"""

#### La lista de tablas se debe obtener del pre-filtro

In [ ]:
preFilterTables = ['USERS', 'CREDIT_CARD', 'DEBIT_CARD', 'ACCOUNT','P_PERSON'] # Ejemplo
tables_preFilter = db.get_table_info(preFilterTables)

In [ ]:
question =  "How much credit does user Gonzalo have in the savings account?"

In [ ]:
  instruction = "#Find the tables for generating SQL queries for each question based on the database schema and Foreign keys.\n"
  prompt = instruction + tables_prompt + 'Q: "' + question + """"\nA: Let’s think step by step."""

In [ ]:
tablesResult = llm(prompt)

El Resultado obtenido es de la forma:

In the question "How much credit does user Gonzalo have in the savings account?", we are asked for information about a user call Gonzalo, the table USER have a foreign_keys from P_PERSON, and P_PERSON have an atribure ACCOUNT that is a foreign_keys from ACCOUNT, so we need the tables USER, P_PERSON and ACCOUNT. 
Relevant_tables=["USERS","P_PERSON","ACCOUNT"]

Para el Constructor solo nos importa la lista de tablas relevantes. Por lo tanto, hay que filtrar la respuesta de la llm.

In [ ]:
tables = tablesResult.split("=")
Relevant_tables= tables[1]